<a href="https://colab.research.google.com/github/jonik2909/computer-vision/blob/main/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision


In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
print("CUDA available:", torch.cuda.is_available())

In [ ]:
scalar = torch.tensor(5) # 0D

print("scalar:", scalar)
print("shape", scalar.shape)
print("Number of Dimensions", scalar.ndim)
print("Size", scalar.size())

In [ ]:
vector = torch.tensor([1,2,3]) # 1D

print("vector:", vector)
print("shape", vector.shape)
print("Number of Dimensions", vector.ndim)
print("Size", vector.size())

In [ ]:
MATRIX = torch.tensor([[1,2,3], [4,5,6]]) # 2D

print("MATRIX:", MATRIX)
print("shape", MATRIX.shape)
print("Number of Dimensions", MATRIX.ndim)
print("Size", MATRIX.size())

In [ ]:
tensor3D = torch.stack([MATRIX, MATRIX + 10]) # 3D

print("tensor3D:", tensor3D)
print("shape", tensor3D.shape) # [2, 2, 3] => [stacks, rows in each matrix (height), columns in each matrix (width)]
print("Number of Dimensions", tensor3D.ndim)
print("Size", tensor3D.size())

In [ ]:
random_numbers = torch.rand(2,3) # 0 ~ 1

print("random_numbers:", random_numbers)
print("Data type", random_numbers.dtype)
print("Device of random_numbers", random_numbers.device)

In [ ]:
print("Zeros:", torch.zeros(3,3))

In [ ]:
print("Ones:", torch.ones(2,4))

# MATH

In [ ]:
x = torch.tensor([[1, 2],
                  [3, 4]])

y = torch.tensor([[5, 6],
                  [7, 8]])

# Element-wise addition
print("Addition:", x + y)

print("=================")

# Element-wise multiplication
print("Multiplication:", x * y)

print("=================")

# Matrix multiplication
print("Matrix @ multiplication:", x @ y)

# AI MODEL

In [ ]:
# Import PyTorch and matplotlib
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks | nn module
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# CNN
# NN

In [ ]:
# Create weight and bias
weight = 0.7 # weight
bias = 0.3 # bias

# Create range values
start = 0
end = 1
step = 0.02

# Create X and y (features and labels)
X = torch.arange(start, end, step).unsqueeze(dim=1) # without unsqueeze, errors will happen later on (shapes within linear layers)
y = weight * X + bias #

# Liner Regression: y = wx + b
print("X:", X)

print("\n y:", y)

# Split data
train_split = int(0.8 * len(X)) # 10000(100%) data => 8000(80%) train | 2000(20%) test
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
  plt.figure(figsize=(10, 7))

  #Plot training data in blue
  plt.scatter(train_data, train_labels, c='b', s=4, label='Training data')

  # Plot test data in green
  plt.scatter(test_data, test_labels, c='g', s=4, label='Testing data')

  if predictions is not None:
    plt.scatter(test_data, predictions, c='r', s=4, label='Predictions')

  plt.legend(prop={'size': 14})

# CALL
plot_predictions()

In [ ]:
# Subclass nn.Module to make our model
class LinearRegressionModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        # Use nn.Linear() for creating the model parameters
        self.linear_layer = nn.Linear(in_features=1, #weight and
                                      out_features=1) # Linear => y = wx+b

    # Define the forward computation (input data x flows through nn.Linear())
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

# Set the manual seed when creating the model (this isn't always needed but is used for demonstrative purposes, try commenting it out and seeing what happens)
torch.manual_seed(42)
model_1 = LinearRegressionModelV2()
model_1, model_1.state_dict(),

In [ ]:
# Create loss function
loss_fn = nn.L1Loss() # Loss Function | MAE => Mean Absolute Error

# Create optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(), # optimize newly created model's parameters
                            lr=0.01) # weight, bias | equal

In [ ]:
# See model's untrained predictions (on test set)
model_1.eval() #

with torch.inference_mode():
    y_preds_early = model_1(X_test) # backwards

plot_predictions(predictions=y_preds_early)
plt.title("Predictions before training (red = random guesses)")


In [ ]:
# Set the number of epochs
epochs = 1000

# Put data on the available device
# Without this, error will happen (not all model/data on device)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

for epoch in range(epochs):
    ### Training
    model_1.train() # train mode is on by default after construction

    # 1. Forward pass
    y_pred = model_1(X_train)

    # 2. Calculate loss
    loss = loss_fn(y_pred, y_train)

    # 3. Zero gradient optimizer
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Step the optimizer
    optimizer.step() # weight and bias

    ### Testing
    model_1.eval() # put the model in evaluation mode for testing (inference)
    # 1. Forward pass
    with torch.inference_mode():
        test_pred = model_1(X_test)

        # 2. Calculate the loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")

In [ ]:
# Turn model into evaluation mode
model_1.eval()

# Make predictions on the test data
with torch.inference_mode():
    y_preds = model_1(X_test)
y_preds

In [ ]:
# Put data on the CPU and plot it
plot_predictions(predictions=y_preds)

In [ ]:
torch.save(model_1.state_dict(), 'trained_model.pth')

In [ ]:
loaded_model = LinearRegressionModelV2()

loaded_model.load_state_dict(torch.load('/content/trained_model.pth'))

loaded_model.eval()